# Import Modules

In [1]:
import os
import pandas as pd
import numpy.random as rnd
from dotenv import load_dotenv
import requests
import warnings
import json
import sqlalchemy as sql
from pathlib import Path
import datetime
import yfinance as yf
import numpy as np


from MCForecastTools import MCSimulation
warnings.filterwarnings('ignore')


%matplotlib inline

#warnings.filterwarnings("ignore")
#pd.options.display.float_format = '{:.4%}'.format

# Client Input Module

# Data Input for Universe (Multiple Asset Classes)

In [2]:
# Date range
today = datetime.date.today()
start_date = today - datetime.timedelta(days=365*5) #trading days(252) * amount of years to go back 

start = start_date.strftime("%Y-%m-%d")
end = today.strftime("%Y-%m-%d")

# Tickers of assets
assets = ["MSFT", "TSLA", "META", "UNH", "JNJ", "JPM", "V", "LLY", "AVGO", "PG", "MA", "HD", "MRK", "NVDA", "AMZN", "BRK-B",
         "GOOG", "XOM", "CVX", "PEP", "COST", "KO", "ABBV", "ADBE", "WMT", "MCD", "CSCO", "CRM"]
assets.sort()

# Downloading data
og_data = yf.download(assets, start = start, end = end)
data = og_data.loc[:,('Adj Close', slice(None))]
data.columns = assets

Y = data[assets].pct_change().dropna()

display(start)
display(end)
display(Y)

[*********************100%***********************]  28 of 28 completed


'2018-06-30'

'2023-06-29'

,ABBV,ADBE,AMZN,AVGO,BRK-B,COST,CRM,CSCO,CVX,GOOG,...,MRK,MSFT,NVDA,PEP,PG,TSLA,UNH,V,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2018-07-03,0.005370,-0.005139,-0.011565,-0.011403,-0.009636,-0.004183,-0.018443,-0.003270,0.003863,-0.021792,...,0.004135,-0.009599,-0.022292,-0.006665,-0.002944,-0.072253,-0.002020,-0.007925,0.005238,0.005871
2018-07-05,0.009294,0.008967,0.003406,0.025534,0.007956,0.009753,0.014002,0.005196,-0.006013,0.019385,...,0.015319,0.007168,0.024869,0.006893,0.016816,-0.005469,0.014252,0.013998,0.001539,0.000851
2018-07-06,0.025826,0.016422,0.006413,0.009291,0.000320,0.002295,0.011662,0.003524,0.001291,0.014142,...,0.009085,0.014034,0.018951,0.000091,0.001263,-0.000841,0.001038,0.006002,-0.000710,0.000243
2018-07-09,0.001238,0.006366,0.016596,-0.001009,0.011090,0.006297,0.002546,-0.000937,0.014903,0.012174,...,-0.002894,0.006821,0.007763,-0.016429,-0.018283,0.031110,0.011605,0.010664,0.016803,0.006923
2018-07-10,0.018549,-0.004564,0.002329,0.012650,0.001371,0.003603,0.004867,0.004453,0.012699,-0.001049,...,0.004515,0.002651,0.016048,0.047606,0.025173,0.012433,0.007372,0.008634,0.014896,0.009168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-22,0.004019,0.000209,0.042618,-0.006170,-0.004873,0.008814,0.017653,0.005310,-0.014463,0.021524,...,0.023030,0.018437,-0.000465,0.010354,0.003413,0.019849,0.004824,0.005016,0.008352,-0.005488
2023-06-23,-0.010844,0.014950,-0.006300,-0.024073,-0.005075,0.000458,-0.015003,-0.015454,-0.008451,-0.006862,...,0.007738,-0.013806,-0.018966,-0.006832,-0.009937,-0.030271,-0.004279,0.005079,-0.001862,-0.008713
2023-06-26,-0.007137,-0.010748,-0.015464,-0.000633,-0.003371,-0.001945,-0.012994,-0.002384,0.017575,-0.031946,...,-0.013263,-0.019163,-0.037362,-0.006342,0.001010,-0.060600,0.004319,-0.014158,-0.002637,0.018457


In [3]:
Y_cumulative_ret = (1 + Y).cumprod()
Y_cumulative_ret

#Set the time period and trading days
time_period = 1
trading_days = 252 * time_period

In [4]:
# Calculate and sort the standard deviation for all assetts
asset_standard_deviation = pd.DataFrame(Y.std(), columns = ['Standard Deviation'], index=assets)
asset_standard_deviation

,Standard Deviation
ABBV,0.016748
ADBE,0.024025
AMZN,0.022962
AVGO,0.023650
BRK-B,0.014257
COST,0.015395
CRM,0.025025
CSCO,0.018155
CVX,0.022463
GOOG,0.020146


In [5]:
# Calculate and sort the annualized standard deviation (252 trading days) 
annual_stdev_calc = Y.std()*np.sqrt(trading_days)
annual_standard_deviation = pd.DataFrame(annual_stdev_calc, columns = ['Annual Standard Deviation'], index=assets)
display(annual_standard_deviation)

,Annual Standard Deviation
ABBV,0.265870
ADBE,0.381391
AMZN,0.364503
AVGO,0.375437
BRK-B,0.226326
COST,0.244380
CRM,0.397253
CSCO,0.288201
CVX,0.356582
GOOG,0.319809


In [6]:
annualized_avg_return_calc = Y.mean() * trading_days
annualized_average_returns = pd.DataFrame(annualized_avg_return_calc, columns = ['Annual Average Returns'], index=assets)
annualized_average_returns

,Annual Average Returns
ABBV,0.153578
ADBE,0.210621
AMZN,0.148599
AVGO,0.357889
BRK-B,0.141300
COST,0.232325
CRM,0.161080
CSCO,0.106819
CVX,0.152435
GOOG,0.204651


In [7]:
# Calculate the annualized Sharpe Ratios
Sharpe_ratios_calc = annualized_average_returns['Annual Average Returns'] / annual_standard_deviation['Annual Standard Deviation']
Sharpe_ratios = pd.DataFrame(Sharpe_ratios_calc, columns = ['Sharpe Ratio'], index=assets)
Sharpe_ratios

,Sharpe Ratio
ABBV,0.577643
ADBE,0.552245
AMZN,0.407676
AVGO,0.953260
BRK-B,0.624320
COST,0.950669
CRM,0.405484
CSCO,0.370642
CVX,0.427489
GOOG,0.639917


In [8]:
# Merge all of the dataframes
Asset_Quant_Data = pd.merge(pd.merge(pd.merge           
                        (asset_standard_deviation, annual_standard_deviation, left_index=True, right_index=True),
                        annualized_average_returns, left_index=True, right_index=True), 
                        Sharpe_ratios, left_index=True, right_index=True)
Asset_Quant_Data

,Standard Deviation,Annual Standard Deviation,Annual Average Returns,Sharpe Ratio
ABBV,0.016748,0.265870,0.153578,0.577643
ADBE,0.024025,0.381391,0.210621,0.552245
AMZN,0.022962,0.364503,0.148599,0.407676
AVGO,0.023650,0.375437,0.357889,0.953260
BRK-B,0.014257,0.226326,0.141300,0.624320
COST,0.015395,0.244380,0.232325,0.950669
CRM,0.025025,0.397253,0.161080,0.405484
CSCO,0.018155,0.288201,0.106819,0.370642
CVX,0.022463,0.356582,0.152435,0.427489
GOOG,0.020146,0.319809,0.204651,0.639917
